In [ ]:
import grass.script as gs
import os
from datetime import datetime

def log_message(message):
    """Helper function for logging with timestamp"""
    print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {message}")

def safe_remove_raster(name):
    """Safely remove raster if it exists"""
    try:
        if name in gs.list_strings(type='raster'):
            gs.run_command('g.remove', type='raster', name=name, flags='f', quiet=True)
            return True
    except:
        pass
    return False

# 1. Verify GRASS environment
log_message("Checking GRASS GIS environment...")
location_info = gs.read_command('g.gisenv', flags='n').strip()
mapset_info = gs.read_command('g.mapset', flags='p').strip()
log_message(f"Current location: {location_info}")
log_message(f"Current mapset: {mapset_info}")

# 2. Define input directory and files (updated to include all years through 2022)
input_dir = '/Users/ghulamabbaszafari/Downloads/rest/reclassification/OneDrive_1_04-04-2025/reclass_data/GLC_FCS30D_reclass/GLC_FCS30D_reclass'
input_files = {
    # 5-year intervals (1985-2000)
    'GLC_FCS30_1985': os.path.join(input_dir, 'GLC_FCS30_1985_mosaic_reclass.tif'),
    'GLC_FCS30_1990': os.path.join(input_dir, 'GLC_FCS30_1990_mosaic_reclass.tif'),
    'GLC_FCS30_1995': os.path.join(input_dir, 'GLC_FCS30_1995_mosaic_reclass.tif'),
    'GLC_FCS30_2000': os.path.join(input_dir, 'GLC_FCS30_2000_mosaic_reclass.tif'),

    # Annual intervals (2001-2022)
    'GLC_FCS30_2001': os.path.join(input_dir, 'GLC_FCS30_2001_mosaic_reclass.tif'),
    'GLC_FCS30_2002': os.path.join(input_dir, 'GLC_FCS30_2002_mosaic_reclass.tif'),
    'GLC_FCS30_2003': os.path.join(input_dir, 'GLC_FCS30_2003_mosaic_reclass.tif'),
    'GLC_FCS30_2004': os.path.join(input_dir, 'GLC_FCS30_2004_mosaic_reclass.tif'),
    'GLC_FCS30_2005': os.path.join(input_dir, 'GLC_FCS30_2005_mosaic_reclass.tif'),
    'GLC_FCS30_2006': os.path.join(input_dir, 'GLC_FCS30_2006_mosaic_reclass.tif'),
    'GLC_FCS30_2007': os.path.join(input_dir, 'GLC_FCS30_2007_mosaic_reclass.tif'),
    'GLC_FCS30_2008': os.path.join(input_dir, 'GLC_FCS30_2008_mosaic_reclass.tif'),
    'GLC_FCS30_2009': os.path.join(input_dir, 'GLC_FCS30_2009_mosaic_reclass.tif'),
    'GLC_FCS30_2010': os.path.join(input_dir, 'GLC_FCS30_2010_mosaic_reclass.tif'),
    'GLC_FCS30_2011': os.path.join(input_dir, 'GLC_FCS30_2011_mosaic_reclass.tif'),
    'GLC_FCS30_2012': os.path.join(input_dir, 'GLC_FCS30_2012_mosaic_reclass.tif'),
    'GLC_FCS30_2013': os.path.join(input_dir, 'GLC_FCS30_2013_mosaic_reclass.tif'),
    'GLC_FCS30_2014': os.path.join(input_dir, 'GLC_FCS30_2014_mosaic_reclass.tif'),
    'GLC_FCS30_2015': os.path.join(input_dir, 'GLC_FCS30_2015_mosaic_reclass.tif'),
    'GLC_FCS30_2016': os.path.join(input_dir, 'GLC_FCS30_2016_mosaic_reclass.tif'),
    'GLC_FCS30_2017': os.path.join(input_dir, 'GLC_FCS30_2017_mosaic_reclass.tif'),
    'GLC_FCS30_2018': os.path.join(input_dir, 'GLC_FCS30_2018_mosaic_reclass.tif'),
    'GLC_FCS30_2019': os.path.join(input_dir, 'GLC_FCS30_2019_mosaic_reclass.tif'),
    'GLC_FCS30_2020': os.path.join(input_dir, 'GLC_FCS30_2020_mosaic_reclass.tif'),
    'GLC_FCS30_2021': os.path.join(input_dir, 'GLC_FCS30_2021_mosaic_reclass.tif'),
    'GLC_FCS30_2022': os.path.join(input_dir, 'GLC_FCS30_2022_mosaic_reclass.tif')
}

# 3. Target projection parameters
target_epsg = "EPSG:3035"  # ETRS89-extended / LAEA Europe
target_resolution = 30  # meters (original resolution)

# 4. Processing workflow
def process_rasters():
    """Main processing workflow"""
    # First clean any existing rasters (only if they exist)
    log_message("Cleaning existing rasters if present...")
    for raster in input_files.keys():
        if safe_remove_raster(raster):
            log_message(f"Removed existing {raster}")

    # Process each file
    for output_name, input_path in input_files.items():
        log_message(f"\nProcessing {output_name}...")

        # Verify input file exists
        if not os.path.exists(input_path):
            log_message(f"Error: Input file not found - {input_path}")
            continue

        # Create temporary output path
        temp_output = f"/tmp/{output_name}_3035.tif"

        # Step 1: Reproject with GDAL
        log_message(f"Reprojecting to {target_epsg} with gdalwarp...")
        cmd = f"gdalwarp -t_srs {target_epsg} -tr {target_resolution} {target_resolution} -r near -overwrite -co COMPRESS=LZW {input_path} {temp_output}"
        ret = os.system(cmd)
        if ret != 0:
            log_message(f"Error in gdalwarp for {output_name}")
            continue

        # Step 2: Import to GRASS
        log_message(f"Importing {output_name} to GRASS...")
        try:
            gs.run_command('r.import',
                          input=temp_output,
                          output=output_name,
                          resolution='value',
                          resolution_value=target_resolution,
                          flags='o',
                          quiet=True)
        except Exception as e:
            log_message(f"Error importing {output_name}: {str(e)}")
            continue

        # Clean up temporary file
        if os.path.exists(temp_output):
            os.remove(temp_output)

        # Verify the imported raster
        try:
            info = gs.parse_command('r.info', map=output_name, flags='g')
            if info:
                log_message(f"Successfully imported {output_name}")
                log_message(f"Resolution: {info.get('nsres', 'N/A')}m x {info.get('ewres', 'N/A')}m")
                log_message(f"Extent: n={info.get('north', 'N/A')} s={info.get('south', 'N/A')} e={info.get('east', 'N/A')} w={info.get('west', 'N/A')}")
            else:
                log_message(f"Warning: Could not verify {output_name}")
        except:
            log_message(f"Warning: Could not verify {output_name}")

    # Final region setting
    log_message("\nSetting computational region...")
    existing_rasters = [r for r in input_files.keys() if r in gs.list_strings(type='raster')]
    if existing_rasters:
        gs.run_command('g.region', raster=','.join(existing_rasters), quiet=True)

    # Final verification
    log_message("\nFinal verification:")
    try:
        region = gs.parse_command('g.region', flags='ug')
        if region:
            log_message(f"Region resolution: {region.get('nsres', 'N/A')} x {region.get('ewres', 'N/A')}")
            log_message(f"Region extent: N={region.get('north', 'N/A')} S={region.get('south', 'N/A')} E={region.get('east', 'N/A')} W={region.get('west', 'N/A')}")
        else:
            log_message("Could not get region information")
    except:
        log_message("Could not get region information")

# Run the processing
process_rasters()

log_message("\nProcessing complete. All rasters should now be properly:")
log_message(f"- Projected in {target_epsg}")
log_message(f"- At consistent {target_resolution}m resolution")
log_message("- Ready for analysis")




